# RDF in Python using RDFLib 

## Imports

In [5]:
# !pip install rdflib --user
import rdflib
from rdflib import Graph, Literal, RDF, RDFS, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
import pandas as pd
import numpy as np
import re 
import requests
from bs4 import BeautifulSoup

## Get the ref on IMDB

### Scrap IMDb

In [37]:
t_list = []

source = requests.get("https://www.imdb.com/chart/top/")
source.raise_for_status()
soup = BeautifulSoup(source.text, "html.parser")
movies = soup.find_all('a')

for m in movies:
    href = m.get('href')
    if href != None:
        s = href.split("/")
        try:
            if s[1] == "title":
                t_list += [s[2]]
        except Exception as e:
            pass

t_list

In [84]:
t_list_add = []

#Top 50 films of each IMDb category
addresses = [
    "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=1&ref_=adv_nxt",
    "https://www.imdb.com/search/title/?genres=animation&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_3",
    "https://www.imdb.com/search/title/?genres=biography&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_4",
    "https://www.imdb.com/search/title/?genres=comedy&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_5",
    "https://www.imdb.com/search/title/?genres=crime&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_6",
    "https://www.imdb.com/search/title/?genres=drama&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_7",
    "https://www.imdb.com/search/title/?genres=family&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_8",
    "https://www.imdb.com/search/title/?genres=fantasy&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_9",
    "https://www.imdb.com/search/title/?genres=film_noir&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_10",
    "https://www.imdb.com/search/title/?genres=history&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_11",
    "https://www.imdb.com/search/title/?genres=horror&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_12",
    "https://www.imdb.com/search/title/?genres=music&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_13",
    "https://www.imdb.com/search/title/?genres=musical&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_14",
    "https://www.imdb.com/search/title/?genres=mystery&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_15",
    "https://www.imdb.com/search/title/?genres=romance&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_16",
    "https://www.imdb.com/search/title/?genres=sci_fi&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_17",
    "https://www.imdb.com/search/title/?genres=sport&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_18",
    "https://www.imdb.com/search/title/?genres=thriller&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_19",
    "https://www.imdb.com/search/title/?genres=western&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_21",
    "https://www.imdb.com/search/title/?genres=western&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_21",
]

for address in addresses:

    source = requests.get(address)
    source.raise_for_status()
    soup = BeautifulSoup(source.text, "html.parser")
    movies = soup.find_all('a')

    for m in movies:
        href = m.get('href')
        if href != None:
            s = href.split("/")
            try:
                if s[1] == "title":
                    t_list_add += [s[2]]
            except Exception as e:
                pass

t_list_add

['tt0468569',
 'tt0468569',
 'tt0468569',
 'tt0167260',
 'tt0167260',
 'tt0167260',
 'tt0120737',
 'tt0120737',
 'tt0120737',
 'tt1375666',
 'tt1375666',
 'tt1375666',
 'tt0167261',
 'tt0167261',
 'tt0167261',
 'tt0133093',
 'tt0133093',
 'tt0133093',
 'tt0080684',
 'tt0080684',
 'tt0080684',
 'tt10189514',
 'tt10189514',
 'tt10189514',
 'tt0076759',
 'tt0076759',
 'tt0076759',
 'tt0103064',
 'tt0103064',
 'tt0103064',
 'tt0047478',
 'tt0047478',
 'tt0047478',
 'tt0056058',
 'tt0056058',
 'tt0056058',
 'tt20850406',
 'tt20850406',
 'tt20850406',
 'tt0172495',
 'tt0172495',
 'tt0172495',
 'tt0110413',
 'tt0110413',
 'tt0110413',
 'tt9900782',
 'tt9900782',
 'tt9900782',
 'tt4154796',
 'tt4154796',
 'tt4154796',
 'tt4154756',
 'tt0082971',
 'tt0082971',
 'tt0082971',
 'tt4154756',
 'tt4154756',
 'tt4154756',
 'tt4633694',
 'tt4633694',
 'tt4633694',
 'tt0364569',
 'tt0364569',
 'tt0364569',
 'tt0090605',
 'tt0090605',
 'tt0090605',
 'tt1345836',
 'tt1345836',
 'tt1345836',
 'tt0119698',


In [89]:
t_list_add = pd.DataFrame(t_list_add).drop_duplicates()[0].tolist()

### Get the film in JSON from OMDb

In [56]:
mykey = "nicekey"
responses = []
for i in t_list:
    responses += [requests.request(url=f"http://www.omdbapi.com/?i={i}&r=json&apikey=6f9b1949",method="GET")]

Additional films run from time to time

In [90]:
mykey = "nicekey"
responses_add = []
for i in t_list_add:
    responses_add += [requests.request(url=f"http://www.omdbapi.com/?i={i}&r=json&apikey=6f9b1949",method="GET")]

In [55]:
responses[0].json()

{'Title': 'The Shawshank Redemption',
 'Year': '1994',
 'Rated': 'R',
 'Released': '14 Oct 1994',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Director': 'Frank Darabont',
 'Writer': 'Stephen King, Frank Darabont',
 'Actors': 'Tim Robbins, Morgan Freeman, Bob Gunton',
 'Plot': 'Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Nominated for 7 Oscars. 21 wins & 43 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '9.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '91%'},
  {'Source': 'Metacritic', 'Value': '81/100'}],
 'Metascore': '81',
 'imdbRating': '9.3',
 'imdbVotes': '2,704,541',
 'imdbID': 'tt0111161',
 'Type': 'movie',
 'DVD': '21 Dec 1999',
 'BoxOffice

### Store them in json files

In [112]:
def concatString(s):
    s = re.split('[-+#,:?"\'!^$& éèç|()€¤$£]', s)
    S = ""
    for i in s:
        S += i
    return S

In [67]:
type(responses[0].json())

dict

In [121]:
for r in responses:
    jsn = pd.DataFrame(r.json())
    filmName = concatString(r.json()["Title"])
    jsn.to_json("./film/"+filmName+".json")

In [122]:
for r in responses_add:
    jsn = pd.DataFrame(r.json())
    filmName = concatString(r.json()["Title"])
    jsn.to_json("./film/"+filmName+".json")

## Graph the current ontology

In [69]:
from rdflib import Graph

In [222]:
#Defining the namespace
nspace = Namespace("Project.rdf")

# Create a Graph
g = Graph()

# Parse in an RDF file hosted locally
g.parse("Project.rdf")

g.namespace_manager.bind("nl",nspace)

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format="n3"))

Graph g has 189 statements.
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .
@prefix untitled-ontology-3: <http://www.semanticweb.org/marya/ontologies/2023/1/untitled-ontology-3#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/marya/ontologies/2023/1/untitled-ontology-3> a owl:Ontology ;
    owl:imports schema: .

untitled-ontology-3:AnaDeArmas a untitled-ontology-3:Female,
        owl:NamedIndividual ;
    untitled-ontology-3:age "34"^^xsd:int ;
    untitled-ontology-3:isActorOf untitled-ontology-3:KnivesOut ;
    untitled-ontology-3:name "Ana de Armas"^^xsd:string ;
    untitled-ontology-3:nationality "Cuba"^^xsd:string .

untitled-ontology-3:ChrisEvans a untitled-ontology-3:Male,
        owl:NamedIndividual ;
    untitled-ontology-3:isActorOf untitled-ontology-3:KnivesOut ;
    untitled-ontology-3:name "Chris Evans"^^xsd:string ;
    untitled-ontology-3:n

## Get a JSON document
When scraping the json, we add all films in different files but a film can be added multiple times to the same file.

In [116]:
film = pd.read_json("./film/12AngryMen.json", encoding='utf-8-sig')

In [127]:
film

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,12 Angry Men,1957,Approved,10 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam",The jury in a New York City murder trial is fr...,...,97,9,"798,687",tt0050083,movie,04 Mar 2008,N/A,"Criterion Collection, MGM/UA Home Entertainmen...",N/A,True
1,12 Angry Men,1957,Approved,10 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam",The jury in a New York City murder trial is fr...,...,97,9,"798,687",tt0050083,movie,04 Mar 2008,N/A,"Criterion Collection, MGM/UA Home Entertainmen...",N/A,True
2,12 Angry Men,1957,Approved,10 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam",The jury in a New York City murder trial is fr...,...,97,9,"798,687",tt0050083,movie,04 Mar 2008,N/A,"Criterion Collection, MGM/UA Home Entertainmen...",N/A,True


## Insert JSON document into graph

In [137]:
""" {} is a greate {}""".format("He", "composer.")

' He is a greate composer.'

In [189]:
iri = "http://www.semanticweb.org/marya/ontologies/2023/1/untitled-ontology-3/"

In [160]:
def actorExsists(g, actor):
    """
    actor is actor name ex : "Bruce Willis"
    """
    nameConcat = concatString(actor)
    res = g.query(f"""
        SELECT *
        WHERE {{ <{iri}{nameConcat}> <{iri}name> \"{actor}\" }} 
    """)
    return res

In [165]:
actor = film.loc[0]["Actors"].split(", ")[0]
actor

'Henry Fonda'

In [203]:
for row in g.query(f"""SELECT ?title {{ ?movie nl:title ?title }}"""):
    print(row)

In [218]:
!pip install graphviz --user


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
create_film = f"INSERT DATA {{ <{iri}{film_id}> a <{iri}Movie>}}"
update_film_title = f"INSERT DATA {{ <{iri}{film_id}> <{iri}title> \"{actor}\" }}"
update_film_yearReleased = f"INSERT DATA {{ <{iri}{film_id}> <{iri}yearReleased> \"{film["Year"]}\" }}"
update_film_title = f"INSERT DATA {{ <{iri}{film_id}> <{iri}title> \"{actor}\" }}"
update_film_title = f"INSERT DATA {{ <{iri}{film_id}> <{iri}title> \"{actor}\" }}"
update_film_title = f"INSERT DATA {{ <{iri}{film_id}> <{iri}title> \"{actor}\" }}"
update_film_actor = f"INSERT DATA {{ <{iri}{film_id}> <{iri}isPlayedBy> <{iri}{film_id}> }}"

In [187]:
for path in os.listdir("./film/"):
    film = pd.read_json(path, encoding='utf-8-sig')
    f1 =  film.loc[0]
    film_id = concatString(f1["Title"])
    actors = f1["Actors"].split(", ")

    for actor in actors:
        actor_id = concatString(actor)
        print(actor_id)
        create_actor = f"INSERT DATA {{ <{iri}{actor_id}> a <{iri}Person>}}"
        update_actor_name = f"INSERT DATA {{ <{iri}{actor_id}> <{iri}name> \"{actor}\" }}"
        
        update_film_actor = f"INSERT DATA {{ <{iri}{film_id}> <{iri}isPlayedBy> <{iri}{actor_id}> }}"

        g.update(create_actor)
        g.update(update_actor_name)
        g.update(update_film_actor)

HenryFonda
LeeJ.Cobb
MartinBalsam


In [188]:
print(g.serialize(format="n3"))

@prefix ns1: <http://www.semanticweb.org/marya/ontologies/2023/1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .
@prefix untitled-ontology-3: <http://www.semanticweb.org/marya/ontologies/2023/1/untitled-ontology-3#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:untitled-ontology-3 a owl:Ontology ;
    owl:imports schema: .

untitled-ontology-3:AnaDeArmas a untitled-ontology-3:Female,
        owl:NamedIndividual ;
    untitled-ontology-3:age "34"^^xsd:int ;
    untitled-ontology-3:isActorOf untitled-ontology-3:KnivesOut ;
    untitled-ontology-3:name "Ana de Armas"^^xsd:string ;
    untitled-ontology-3:nationality "Cuba"^^xsd:string .

untitled-ontology-3:ChrisEvans a untitled-ontology-3:Male,
        owl:NamedIndividual ;
    untitled-ontology-3:isActorOf untitled-ontology-3:KnivesOut ;
    untitled-ontology-3:name "Chris Evans"^^xsd:string ;
    untitled-ontology-3:nationali

In [154]:
for row in actorExsists(g, "Ana de Armas"):
    print(row)

In [179]:
def kInG(k, g):
    for s, p, o in g:
        if k == s or k == p or k == o:
            return True
    return False

In [180]:
def addActors(g, actor_list, film_name):
    """
    g is the graph
    actor_list of form : [actor_name_1, ..., actor_name_n]
    add each actor as an actor with name actor_name_i
    """
    for actor in actor_list:
        firstname = actor.split(" ")[0]
        lastname = actor.split(" ")[1]
        actor_name = firstname+"_"+lastname
        adr = "untitled-ontology-3:"+str(actor_name)
        if not kInG(adr, g):
            a = URIRef(adr)
            #add rdf triples
            g.add((a, RDF.type, nspace.Person))
            g.add((a, nspace.name, Literal(actor)))
            g.add((a, nspace.isActorOf, nspace.Movie))
            print(f"Actor {actor_name} added")

In [1]:
addActors(g, film.loc["Actors"].to_list()[0].split(", "), film.loc["Title"][0])

NameError: name 'addActors' is not defined

In [182]:
def addfilms(g, film):
    """
    g is the graph
    film of form : dataframe with all infos (from json)
    """

    film_title = concatString(film["title"])
    adr = "untitled-ontology-3:"+film_title

    

        firstname = actor.split(" ")[0]
        lastname = actor.split(" ")[1]
        actor_name = firstname+"_"+lastname
        
        if not kInG(adr, g):
            a = URIRef(adr)
            #add rdf triples
            g.add((a, RDF.type, nspace.Person))
            g.add((a, nspace.name, Literal(actor)))
            g.add((a, nspace.isActorOf, Literal(film_name)))
            print(f"Actor {actor_name} added")

IndentationError: unexpected indent (105303359.py, line 12)

In [ ]:
s = "hello my name is Baptiste"

S

'hellomynameisBaptiste'

In [183]:
print(g.serialize(format="n3"))

@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .
@prefix untitled-ontology-3: <http://www.semanticweb.org/marya/ontologies/2023/1/untitled-ontology-3#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/marya/ontologies/2023/1/untitled-ontology-3> a owl:Ontology ;
    owl:imports schema: .

untitled-ontology-3:DanielCraig a untitled-ontology-3:Male,
        owl:NamedIndividual ;
    untitled-ontology-3:age "55"^^xsd:int ;
    untitled-ontology-3:isActorOf untitled-ontology-3:CasinoRoyale ;
    untitled-ontology-3:name "Daniel Craig"^^xsd:string .

untitled-ontology-3:JohnTravolta a untitled-ontology-3:Male,
        owl:NamedIndividual ;
    untitled-ontology-3:age "59"^^xsd:int ;
    untitled-ontology-3:isActorOf untitled-ontology-3:PulpFiction ;
    untitled-ontology-3:name "John Travolta"^^xsd:string ;
    untitled-ontology-3:nationality "American"^^xsd:string

In [ ]:
import rdflib

conj=rdflib.ConjunctiveGraph()

NS=rdflib.Namespace("http://example.com/#")
NS_CTX=rdflib.Namespace("http://example.com/context/#")

alice=NS.alice
bob=NS.bob
charlie=NS.charlie

pizza=NS.pizza
meat=NS.meat
chocolate=NS.chocolate

loves=NS.loves
hates=NS.hates
likes=NS.likes
dislikes=NS.dislikes

love_ctx=Graph(conj.store, NS_CTX.love)
food_ctx=Graph(conj.store, NS_CTX.food)

love_ctx.add( (alice, loves, bob) )
love_ctx.add( (alice, loves, charlie) )
love_ctx.add( (bob, hates, charlie) )
love_ctx.add( (charlie, loves, bob) )

food_ctx.add( (alice, likes, chocolate) )
food_ctx.add( (alice, likes, meat) )
food_ctx.add( (alice, dislikes, pizza) )

print("Full context")
for t in conj:
    print (t)

print( "")
print( "Contexts")
for c in conj.contexts():
    print (c)

print( "love context")
for t in love_ctx:
    print (t)

print("food context")
for t in food_ctx:
    print (t)

Full context
(rdflib.term.URIRef('http://example.com/#charlie'), rdflib.term.URIRef('http://example.com/#loves'), rdflib.term.URIRef('http://example.com/#bob'))
(rdflib.term.URIRef('http://example.com/#alice'), rdflib.term.URIRef('http://example.com/#loves'), rdflib.term.URIRef('http://example.com/#charlie'))
(rdflib.term.URIRef('http://example.com/#alice'), rdflib.term.URIRef('http://example.com/#likes'), rdflib.term.URIRef('http://example.com/#chocolate'))
(rdflib.term.URIRef('http://example.com/#alice'), rdflib.term.URIRef('http://example.com/#loves'), rdflib.term.URIRef('http://example.com/#bob'))
(rdflib.term.URIRef('http://example.com/#alice'), rdflib.term.URIRef('http://example.com/#likes'), rdflib.term.URIRef('http://example.com/#meat'))
(rdflib.term.URIRef('http://example.com/#bob'), rdflib.term.URIRef('http://example.com/#hates'), rdflib.term.URIRef('http://example.com/#charlie'))
(rdflib.term.URIRef('http://example.com/#alice'), rdflib.term.URIRef('http://example.com/#dislik

In [ ]:
conj=rdflib.ConjunctiveGraph()

NS=rdflib.Namespace("Project.rdf")
NS_CTX=rdflib.Namespace("Project.rdf")

alice=NS.alice
bob=NS.bob
charlie=NS.charlie

pizza=NS.pizza
meat=NS.meat
chocolate=NS.chocolate

loves=NS.isActorOf
hates=NS.hates
likes=NS.likes
dislikes=NS.dislikes

love_ctx=Graph(conj.store, NS_CTX.love)
food_ctx=Graph(conj.store, NS_CTX.food)

love_ctx.add( (alice, loves, bob) )
love_ctx.add( (alice, loves, charlie) )
love_ctx.add( (bob, hates, charlie) )
love_ctx.add( (charlie, loves, bob) )

food_ctx.add( (alice, likes, chocolate) )
food_ctx.add( (alice, likes, meat) )
food_ctx.add( (alice, dislikes, pizza) )

print("Full context")
for t in conj:
    print (t)

print( "")
print( "Contexts")
for c in conj.contexts():
    print (c)

print( "love context")
for t in love_ctx:
    print (t)

print("food context")
for t in food_ctx:
    print (t)

Full context
(rdflib.term.URIRef('Project.rdfbob'), rdflib.term.URIRef('Project.rdfhates'), rdflib.term.URIRef('Project.rdfcharlie'))
(rdflib.term.URIRef('Project.rdfalice'), rdflib.term.URIRef('Project.rdflikes'), rdflib.term.URIRef('Project.rdfmeat'))
(rdflib.term.URIRef('Project.rdfalice'), rdflib.term.URIRef('Project.rdflikes'), rdflib.term.URIRef('Project.rdfchocolate'))
(rdflib.term.URIRef('Project.rdfalice'), rdflib.term.URIRef('Project.rdfisActorOf'), rdflib.term.URIRef('Project.rdfbob'))
(rdflib.term.URIRef('Project.rdfalice'), rdflib.term.URIRef('Project.rdfdislikes'), rdflib.term.URIRef('Project.rdfpizza'))
(rdflib.term.URIRef('Project.rdfalice'), rdflib.term.URIRef('Project.rdfisActorOf'), rdflib.term.URIRef('Project.rdfcharlie'))
(rdflib.term.URIRef('Project.rdfcharlie'), rdflib.term.URIRef('Project.rdfisActorOf'), rdflib.term.URIRef('Project.rdfbob'))

Contexts
<Project.rdffood> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory'].
<Project.rdflove> a rdfg:Gra